In [1]:
with open('wnut17train.conll','r') as f:
    train=f.readlines()
#reading train files

In [2]:
from more_itertools import split_after
import pprint
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from functools import partial
import time
pp = pprint.PrettyPrinter()

In [3]:
device= torch.device("cuda")
print(device) 

cuda


In [4]:
sent=[]
splits=list(split_after(train,lambda x:x=="\t\n" or x=='\n'))
for i in splits:
    s=[]
    for j in i[:-1]:
        word,label=j.strip('\n').split('\t')      
        tup=(word,label)
        s.append(tup)
    sent.append(s)
len(sent)  

3394

In [5]:
def sent2labels(sent):
    return [label for token,label in sent]
train_labels=[sent2labels(s) for s in sent]
def sent2tokens(sent):
    return [token.lower() for token,label in sent]
train_sents=[sent2tokens(s) for s in sent]
print(train_sents[1])
print(train_labels[1])

['from', 'green', 'newsfeed', ':', 'ahfa', 'extends', 'deadline', 'for', 'sage', 'award', 'to', 'nov', '.', '5', 'http://tinyurl.com/24agj38']
['O', 'O', 'O', 'O', 'B-group', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [8]:
train_id_2_label = list(set([label for sent in train_labels for label in sent]))
train_label_2_id = {label:i for i, label in enumerate(train_id_2_label)}
print("Number of unique labels in training data:", len(train_id_2_label))


Number of unique labels in training data: 13


In [9]:
def convert_labels_to_inds(sent_labels, label_2_id):
    return [label_2_id[label] for label in sent_labels]
train_label_inds = [convert_labels_to_inds(sent_labels, train_label_2_id) for sent_labels in train_labels]

In [16]:
train_label_2_id #checking the labels

{'B-corporation': 3,
 'B-creative-work': 0,
 'B-group': 1,
 'B-location': 10,
 'B-person': 12,
 'B-product': 11,
 'I-corporation': 6,
 'I-creative-work': 2,
 'I-group': 8,
 'I-location': 9,
 'I-person': 5,
 'I-product': 7,
 'O': 4}

In [6]:
texts=[j for i in train_sents for j in i]
print(len(texts))
texts=["<pad>","<unk>"]+texts #added pad and unk in the beginning
id_2_word=set(texts) #taking the unique words from list
print(len(id_2_word))
id_2_word=list(id_2_word)
word_2_id = {w:i for i,w in enumerate(id_2_word)}#making the word dictionary
instance = train_sents[1]
print(instance)
def convert_tokens_to_inds(sentence, word_2_id):
    return [word_2_id.get(t, word_2_id["<unk>"]) for t in sentence]
token_inds = convert_tokens_to_inds(instance, word_2_id)
pp.pprint(token_inds)


62730
12842
['from', 'green', 'newsfeed', ':', 'ahfa', 'extends', 'deadline', 'for', 'sage', 'award', 'to', 'nov', '.', '5', 'http://tinyurl.com/24agj38']
[9006,
 11938,
 6254,
 12055,
 9301,
 11594,
 3880,
 2476,
 295,
 2640,
 11554,
 8354,
 5738,
 185,
 551]


In [7]:
print([id_2_word[tok_idx] for tok_idx in token_inds])
def pad_sentence_for_window(sentence, window_size, pad_token="<pad>"):
    return [pad_token]*window_size + sentence + [pad_token]*window_size

['from', 'green', 'newsfeed', ':', 'ahfa', 'extends', 'deadline', 'for', 'sage', 'award', 'to', 'nov', '.', '5', 'http://tinyurl.com/24agj38']


In [9]:
window_size = 2
instance = pad_sentence_for_window(train_sents[0], window_size)
print(instance)

['<pad>', '<pad>', '@paulwalk', 'it', "'s", 'the', 'view', 'from', 'where', 'i', "'m", 'living', 'for', 'two', 'weeks', '.', 'empire', 'state', 'building', '=', 'esb', '.', 'pretty', 'bad', 'storm', 'here', 'last', 'evening', '.', '<pad>', '<pad>']


In [10]:
for sent in train_sents[:3]:
    tok_idxs = convert_tokens_to_inds(pad_sentence_for_window(sent, window_size), word_2_id)
    print([id_2_word[idx] for idx in tok_idxs])
#checked with first 3 sentences how does it look after padding

['<pad>', '<pad>', '@paulwalk', 'it', "'s", 'the', 'view', 'from', 'where', 'i', "'m", 'living', 'for', 'two', 'weeks', '.', 'empire', 'state', 'building', '=', 'esb', '.', 'pretty', 'bad', 'storm', 'here', 'last', 'evening', '.', '<pad>', '<pad>']
['<pad>', '<pad>', 'from', 'green', 'newsfeed', ':', 'ahfa', 'extends', 'deadline', 'for', 'sage', 'award', 'to', 'nov', '.', '5', 'http://tinyurl.com/24agj38', '<pad>', '<pad>']
['<pad>', '<pad>', 'pxleyes', 'top', '50', 'photography', 'contest', 'pictures', 'of', 'august', '2010', '...', 'http://bit.ly/bgcyz0', '#photography', '<pad>', '<pad>']


In [10]:
def my_collate(data, window_size, word_2_id):
    """
    For some chunk of sentences and labels
        -add winow padding
        -pad for lengths using pad_sequence
        -convert our labels to one-hots
        -return padded inputs, one-hot labels, and lengths
    """
    
    x_s, y_s = zip(*data)

    # deal with input sentences as we've seen
    window_padded = [convert_tokens_to_inds(pad_sentence_for_window(sentence, window_size), word_2_id)
                                                                                  for sentence in x_s]
    # append zeros to each list of token ids in batch so that they are all the same length
    padded = nn.utils.rnn.pad_sequence([torch.LongTensor(t) for t in window_padded], batch_first=True)
    
    # convert labels to one-hots
    labels = []
    lengths = []
    for y in y_s:
        lengths.append(len(y))
        one_hot = torch.zeros(len(y), len(train_id_2_label))
        y = torch.tensor(y)
        y = y.unsqueeze(1)
        label = one_hot.scatter_(1, y, 1)        
        labels.append(label)
    padded_labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)
    
    return padded.long(), padded_labels, torch.LongTensor(lengths)

class SoftmaxWordWindowClassifier(nn.Module):
    """
    A one-layer, binary word-window classifier.
    """
    def __init__(self, config, vocab_size, pad_idx=0):
        super(SoftmaxWordWindowClassifier, self).__init__()
        """
        Instance variables.
        """
        self.window_size = 2*config["half_window"]+1
        self.embed_dim = config["embed_dim"]
        self.hidden_dim = config["hidden_dim"]
        self.num_classes = config["num_classes"]
        self.freeze_embeddings = config["freeze_embeddings"]
        
        """
        Embedding layer
        -model holds an embedding for each layer in our vocab
        -sets aside a special index in the embedding matrix for padding vector (of zeros)
        -by default, embeddings are parameters (so gradients pass through them)
        """
        self.embed_layer = nn.Embedding(vocab_size, self.embed_dim, padding_idx=pad_idx)
        if self.freeze_embeddings:
            self.embed_layer.weight.requires_grad = False
        
        """
        Hidden layer
        -we want to map embedded word windows of dim (window_size+1)*self.embed_dim to a hidden layer.
        -nn.Sequential allows you to efficiently specify sequentially structured models
            -first the linear transformation is evoked on the embedded word windows
            -next the nonlinear transformation tanh is evoked.
        """
        self.hidden_layer = nn.Sequential(nn.Linear(self.window_size*self.embed_dim, 
                                                    self.hidden_dim), 
                                          nn.Tanh())
        
        """
        Output layer
        -we want to map elements of the output layer (of size self.hidden dim) to a number of classes.
        """
        self.output_layer = nn.Linear(self.hidden_dim, self.num_classes)
        
        """
        Softmax
        -The final step of the softmax classifier: mapping final hidden layer to class scores.
        -pytorch has both logsoftmax and softmax functions (and many others)
        -since our loss is the negative LOG likelihood, we use logsoftmax
        -technically you can take the softmax, and take the log but PyTorch's implementation
         is optimized to avoid numerical underflow issues.
        """
        self.log_softmax = nn.LogSoftmax(dim=2)
        
    def forward(self, inputs):
        """
        Let B:= batch_size
            L:= window-padded sentence length
            D:= self.embed_dim
            S:= self.window_size
            H:= self.hidden_dim
            
        inputs: a (B, L) tensor of token indices
        """
        B, L = inputs.size()
        
        """
        Reshaping.
        Takes in a (B, L) LongTensor
        Outputs a (B, L~, S) LongTensor
        """
        # Fist, get our word windows for each word in our input.
        token_windows = inputs.unfold(1, self.window_size, 1)
        _, adjusted_length, _ = token_windows.size()
        
        # Good idea to do internal tensor-size sanity checks, at the least in comments!
        assert token_windows.size() == (B, adjusted_length, self.window_size)
        
        """
        Embedding.
        Takes in a torch.LongTensor of size (B, L~, S) 
        Outputs a (B, L~, S, D) FloatTensor.
        """
        embedded_windows = self.embed_layer(token_windows)
        
        """
        Reshaping.
        Takes in a (B, L~, S, D) FloatTensor.
        Resizes it into a (B, L~, S*D) FloatTensor.
        -1 argument "infers" what the last dimension should be based on leftover axes.
        """
        embedded_windows = embedded_windows.view(B, adjusted_length, -1)
        
        """
        Layer 1.
        Takes in a (B, L~, S*D) FloatTensor.
        Resizes it into a (B, L~, H) FloatTensor
        """
        layer_1 = self.hidden_layer(embedded_windows)
        
        """
        Layer 2
        Takes in a (B, L~, H) FloatTensor.
        Resizes it into a (B, L~, 2) FloatTensor.
        """
        output = self.output_layer(layer_1)
        
        """
        Softmax.
        Takes in a (B, L~, 2) FloatTensor of unnormalized class scores.
        Outputs a (B, L~, 2) FloatTensor of (log-)normalized class scores.
        """
        output = self.log_softmax(output)
        
        return output

def loss_function(outputs, labels, lengths):
    """Computes negative LL loss on a batch of model predictions."""
    B, L, num_classes = outputs.size()
    num_elems = lengths.sum().float()
        
    # get only the values with non-zero labels
    loss = outputs*labels
    
    # rescale average
    return -loss.sum() / num_elems



In [11]:
def train_epoch(loss_function, optimizer, model, train_data):
    
    ## For each batch, we must reset the gradients
    ## stored by the model.   
    total_loss = 0
    for batch, labels, lengths in train_data:
        # clear gradients
        optimizer.zero_grad()
        # evoke model in training mode on batch
        batch=batch.to(device) #sending batch to GPU
        labels=labels.to(device) #sending labels to GPU
        outputs = model.forward(batch)
        # compute loss w.r.t batch
        loss = loss_function(outputs, labels, lengths)
        # pass gradients back, startiing on loss value
        loss.backward()
        # update parameters
        optimizer.step()
        total_loss += loss.item()
    
    # return the total to keep track of how you did this time around
    return total_loss
    

In [20]:
config = {"batch_size": 20,
          "half_window": 2,
          "embed_dim": 25,
          "hidden_dim": 25,
          "num_classes": 13,
          "freeze_embeddings": False,
         }
learning_rate = .0002
num_epochs = 10000
model = SoftmaxWordWindowClassifier(config, len(word_2_id))
model=model.to(device) #sending the model to gpu
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [21]:
train_loader = torch.utils.data.DataLoader(list(zip(train_sents, train_label_inds)), 
                                           batch_size=20, 
                                           shuffle=True, 
                                           collate_fn=partial(my_collate, window_size=2, word_2_id=word_2_id))

In [22]:
for batched_input, batched_labels, batch_lengths in train_loader:
    pp.pprint(("inputs", batched_input, batched_input.size()))
    pp.pprint(("labels", batched_labels, batched_labels.size()))
    pp.pprint(batch_lengths)
    break

('inputs',
 tensor([[ 1603,  1603,  4041,  4293,  6975,   140,  1473, 12173,   989,  7817,
          5738, 11337,  9122,  9628, 10355,  9006, 11337,  8597,  2474, 11976,
          1603,  1603,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0],
        [ 1603,  1603,  9296,   127,  9747,   989,  8556, 11818,  1478,  2681,
         12288,  1357,  2217, 10500,  7173,  2433,    57,  3815,  9609,  7004,
          2681, 12736,  3815,  9747,   989,  2049,  8556,  2474,  9489, 11329,
         12241,  2681,  9385,  8936, 12695,  5444,  3116,  1603,  1603,     0,
             0,     0],
        [ 1603,  1603, 11110,  4586, 12055, 11735,  8665,  6908,  4082,   389,
          7809,  8665,  7453, 10230, 11168, 10682,  3435,  1603,  1603,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,

In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [18]:
model_save_name = 'new_chanegd.pt'
path = F"/content/gdrive/My Drive/{model_save_name}" 

In [ ]:
model.load_state_dict(torch.load(path))
model=model.to(device)#loaded checkpoint, (if needed)

In [23]:
losses = []
start=time.time()
for epoch in range(num_epochs):
    epoch_loss = train_epoch(loss_function, optimizer, model, train_loader)
    if epoch % 1000 == 0:
        losses.append(epoch_loss)
        print(epoch,epoch_loss,time.time()-start)
        torch.save(model.state_dict(), path)
print(losses)

0 412.2885801792145 0.46819400787353516
1000 52.78933657705784 418.63928294181824
2000 51.31286019086838 836.0656394958496
3000 50.7664954662323 1256.3384623527527
4000 49.983798794448376 1673.8972594738007
5000 49.204135581851006 2091.9865374565125
6000 48.79763310402632 2508.4805340766907
7000 48.09832403063774 2924.475048303604
8000 47.757167249917984 3340.0251128673553
9000 47.35470013320446 3753.5658218860626
[412.2885801792145, 52.78933657705784, 51.31286019086838, 50.7664954662323, 49.983798794448376, 49.204135581851006, 48.79763310402632, 48.09832403063774, 47.757167249917984, 47.35470013320446]


In [57]:
with open('emerging.test.conll','r') as f:
    test=f.readlines()
def form(train):    
    sent=[]
    splits=list(split_after(train,lambda x:x=="\t\n" or x=='\n'))
    for i in splits:
        s=[]
        for j in i[:-1]:
            word,label=j.strip('\n').split('\t')      
            tup=(word,label)
            s.append(tup)
        sent.append(s)
    return sent  
sent=form(test)
print(len(sent))
def sent2labels(sent):
    return [label for token,label in sent]
test_labels=[sent2labels(s) for s in sent]
def sent2tokens(sent):
    return [token.lower() for token,label in sent]
test_sents=[sent2tokens(s) for s in sent]


1287


In [63]:
for h,i in enumerate(test_labels):
    for k,j in enumerate(i):
        if len(str(j).split(','))>1:
            a=str(test_labels[h][k]).split(',')
            del a[1:] #since there were few test cases with tokens of multiple labels
            test_labels[h][k]=a[0]


In [66]:
test_label_inds = [convert_labels_to_inds(s, train_label_2_id) for s in test_labels]

In [67]:
test_loader = torch.utils.data.DataLoader(list(zip(test_sents, test_label_inds)), 
                                           batch_size=20, 
                                           shuffle=False, 
                                           collate_fn=partial(my_collate, window_size=2, word_2_id=word_2_id))

In [68]:
test_outputs=[]
for test_instance, labs, _ in test_loader:
    test_instance=test_instance.to(device)
    outputs_full = model.forward(test_instance)
    outputs = torch.argmax(outputs_full, dim=2)
    for i in range(outputs.size(0)):
        test_outputs.append(outputs[i].tolist())  

In [69]:
y_test = test_labels
y_pred = []
for test, pred in zip(test_labels, test_outputs):
    y_pred.append([train_id_2_label[id] for id in pred[:len(test)]])

assert len(y_pred) == len(y_test), '{} vs. {}'.format(len(y_pred), len(y_test))
for i, pred, test in zip(list(range(len(y_pred))), y_pred, y_test):
    assert len(pred) == len(test), '{}: {} vs. {}'.format(i, len(pred), len(test))


In [70]:
# evaluate CRF model
!pip install sklearn-crfsuite
from sklearn_crfsuite import metrics

metrics.flat_f1_score(y_test, y_pred, average='weighted', labels=train_id_2_label)

     |████████████████████████████████| 747kB 25.1MB/s 


0.9073913422398925

In [71]:
# group B and I results
sorted_labels = sorted(
    train_id_2_label,
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

                 precision    recall  f1-score   support

              O      0.938     1.000     0.968     21934
  B-corporation      0.000     0.000     0.000       117
  I-corporation      0.000     0.000     0.000        24
B-creative-work      0.000     0.000     0.000       231
I-creative-work      0.000     0.000     0.000       237
        B-group      0.000     0.000     0.000        97
        I-group      0.000     0.000     0.000        39
     B-location      0.000     0.000     0.000       122
     I-location      0.000     0.000     0.000        39
       B-person      0.000     0.000     0.000       355
       I-person      0.000     0.000     0.000       101
      B-product      0.000     0.000     0.000        56
      I-product      0.000     0.000     0.000        42

       accuracy                          0.938     23394
      macro avg      0.072     0.077     0.074     23394
   weighted avg      0.879     0.938     0.907     23394



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
